In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_hdf('data/train_final.h5')

In [3]:
df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,sin_time_of_day,cos_time_of_day,sin_day_of_week,cos_day_of_week,sin_month,cos_month,year
0,35.0,-0.279847,0.701283,3.012543,1.105701,0.333333,0.831470,0.555570,7.818315e-01,0.623490,8.660254e-01,0.500000,0.833333
1,5.7,-0.246209,-0.542171,-0.447212,-0.693727,0.000000,-0.207912,-0.978148,-7.818315e-01,0.623490,1.224647e-16,-1.000000,0.166667
2,5.7,-0.256903,0.686985,-0.495230,0.844110,0.000000,-0.713250,0.700909,-7.818315e-01,0.623490,-8.660254e-01,0.500000,0.333333
3,6.0,-0.650984,-1.060208,-0.935605,-1.094218,0.000000,0.279829,-0.960050,-2.449294e-16,1.000000,-5.000000e-01,0.866025,0.500000
4,10.5,4.280888,-2.989183,4.009854,-2.401387,0.000000,-0.342020,0.939693,-9.749279e-01,-0.222521,8.660254e-01,0.500000,0.166667


In [4]:
from sklearn.linear_model import LinearRegression

In [7]:
target = df['fare_amount']
df.drop(columns = ['fare_amount'], inplace = True)

In [8]:
regr = LinearRegression() 
regr.fit(df, target) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
dfff = pd.read_csv('data/test.csv')

In [10]:
dfff.to_hdf(r'data/test_smaller.h5', 'data')

In [11]:
import datetime

def minutes_after_midnight(date_time):
    time = date_time.split(' ')
    if len(time) == 3:
        time_arr = time[1].split(':')
        minutes_after_midnight = 60 * int(time_arr[0]) + int(time_arr[1])
        return 2 * np.pi * minutes_after_midnight / 1440
    else:
        return np.nan

def day_of_week(date_time):
    time = date_time.split(' ')
    if len(time) == 3:
        date_arr = time[0].split('-')
        date = datetime.date(int(date_arr[0]), int(date_arr[1]), int(date_arr[2]))
        return 2 * np.pi * (date.weekday() + 1) / 7
    else:
        return np.nan

def month(date_time):
    time = date_time.split(' ')
    if len(time) == 3:
        return 2 * np.pi * int(time[0].split('-')[1]) / 12
    else:
        return np.nan

def year(date_time):
    time = date_time.split(' ')
    if len(time) == 3:
        return float(time[0].split('-')[0])
    else:
        return np.nan

time_of_day_vec = np.vectorize(minutes_after_midnight)
day_of_week_vec = np.vectorize(day_of_week)
month_vec = np.vectorize(month)
year_vec = np.vectorize(year)

def preprocess(df):
    # Drop all null values
    df = df.dropna()

    # Cyclise time and remove key column
    time_column = df['pickup_datetime'].to_numpy()
    df = df.drop(columns=['pickup_datetime', 'key'])

    time_of_day = time_of_day_vec(time_column)
    day_of_week = day_of_week_vec(time_column)
    month = month_vec(time_column)
    year = year_vec(time_column)

    df['sin_time_of_day'] = np.sin(time_of_day)
    df['cos_time_of_day'] = np.cos(time_of_day)
    df['sin_day_of_week'] = np.sin(day_of_week)
    df['cos_day_of_week'] = np.cos(day_of_week)
    df['sin_month'] = np.sin(month)
    df['cos_month'] = np.cos(month)
    df['year'] = year
    return df

In [12]:
dfff = preprocess(dfff)

In [13]:
dfff.to_hdf(r'data/test_smaller.h5', 'data')